In [35]:
import os
import jstyleson
from multiprocessing import Pool
import torch
import dgl
import dgl.data as dgldata
from dgl.data import DGLDataset, CSVDataset, KarateClubDataset, QM7bDataset
import networkx as nx
import sys
import random

In [36]:
sys.path.append("..")
from hypergraph import DiHypergraph  # TODO 包引用，解决路径问题
from utils import load_par  # TODO
from main import load_design

In [37]:

class HypergraphDataset(DGLDataset):
    def __init__(
        self,
        name: str,
        hg_list: list[DiHypergraph],
        par_file_list: list[str],
        multiprocess=1,
        raw_dir="../data",
        save_dir="../data",
        verbose=False
        # force_reload=False,
    ):
        self.hg_list = hg_list
        self.par_file_list = par_file_list
        self.multiprocess = multiprocess
        self.graphs = []
        self.labels = []
        self.graph_names = [hg.design for hg in hg_list]
        super(HypergraphDataset, self).__init__(name, raw_dir=raw_dir, save_dir=save_dir, verbose=verbose)

    def process(self):
        # 将原始数据处理为图、标签和数据集划分的掩码
        print("process")
        self.graphs, self.graph_names = self.generate_graphs()
        self.generate_mask()  # TODO train, valid, test mask
        self.labels = self.generate_labels()

    def __getitem__(self, idx):
        # 通过idx得到与之对应的一个样本
        assert idx < len(self.graphs), "HypergraphLoader: index out of bounds"
        return self.graphs[idx]

    def __len__(self):
        # 数据样本的数量
        return len(self.graphs)

    def save(self):
        # 将处理后的数据保存至 `self.save_path`
        graph_path = os.path.join(self.save_path, self.save_name + ".bin")
        print(f"save {graph_path}")
        dgldata.save_graphs(str(graph_path), self.graphs, {"labels": self.labels})

    def load(self):
        # 从 `self.save_path` 导入处理后的数据
        graph_path = os.path.join(self.save_path, self.save_name + ".bin")
        print(f"load {graph_path}")
        graphs, label_dict = dgldata.load_graphs(graph_path)
        self.graphs = graphs
        self.labels = label_dict["labels"]

    def has_cache(self):
        # 检查在 `self.save_path` 中是否存有处理后的数据
        graph_path = os.path.join(self.save_path, self.save_name + ".bin")
        return os.path.exists(graph_path)

    @property
    def num_partition(self):
        return 2

    @property
    def feature_length(self):
        return 128

    @property
    def save_name(self):
        return self.name + f"_{self.num_partition}_{self.feature_length}_dgl_hypergraph"

    def _generate_one_graph(self, hg):
        print(f"generate {hg.design}")
        # g = nx.DiGraph()
        g = dgl.DGLGraph()
        edge_dict = dict()
        for n1 in range(hg.num_node):
            nei_node, nei_weight = hg.find_neighbors(n1, True)
            for n2, w in zip(nei_node, nei_weight):
                edge_dict[(n1, n2)] = w
            nei_node, nei_weight = hg.find_neighbors(n1, False)
            for n2, w in zip(nei_node, nei_weight):
                edge_dict[(n2, n1)] = w
        h, t, w = [], [], []
        for (n1, n2), wi in edge_dict.items():
            h.append(n1)
            t.append(n2)
            w.append(wi)
        g.add_edges(h, t, {"w": torch.FloatTensor(w)})
        g = g.int()

        # # load label
        # par = load_par(par_file)
        # g.ndata['label']=torch.IntTensor(par)
        return g, hg.design

    def generate_graphs(self):
        print("generate graphs")
        graphs = []
        graph_names = []
        res_list = []
        if self.multiprocess > 1:
            from multiprocessing import Pool

            pool = Pool(8)
            for hg in self.hg_list:
                res_list.append(pool.apply_async(self._generate_one_graph, args=(hg,)))
            pool.close()
            pool.join()
            res_list = [res.get() for res in res_list]
        else:
            for hg in self.hg_list:
                res_list.append(self._generate_one_graph(hg))

        for res in res_list:
            g, gname = res
            graphs.append(g)
            graph_names.append(gname)
        return graphs, graph_names

    def generate_labels(self):
        print(f"generate labels")
        labels = []
        for par_file in self.par_file_list:
            par = load_par(par_file)
            labels.append(torch.IntTensor(par))
        labels = torch.vstack(labels)
        return labels

    def generate_mask(self):
        prev_state = random.getstate()
        torch.manual_seed(3407)
        for gi in self.graphs:
            gi: dgl.DGLGraph
            n = gi.num_nodes()
            train_mask = torch.zeros(n, dtype=torch.bool)
            valid_mask = torch.zeros(n, dtype=torch.bool)
            test_mask = torch.zeros(n, dtype=torch.bool)
            all_node = set(range(n))
            train_idx = random.sample(all_node, int(n * 0.7))
            train_mask[train_idx] = True
            all_node.difference_update(train_idx)
            valid_idx = random.sample(all_node, int(n * 0.2))
            valid_mask[valid_idx] = True
            all_node.difference_update(valid_idx)
            test_idx = list(all_node)
            test_mask[test_idx] = True
            gi.ndata["train_mask"] = train_mask
            gi.ndata["valid_mask"] = valid_mask
            gi.ndata["test_mask"] = test_mask
        random.setstate(prev_state)


In [38]:
benchmark = "ispd2005"
b_pth = os.path.join("..", "res", benchmark)
config_file = os.path.join("..", "par_config", benchmark, "config.json")
with open(config_file, encoding="utf8") as f:
    config = jstyleson.load(f)
# load hypergraph
num_thread = 8
pool = Pool(num_thread)
hg_list: list[DiHypergraph] = []
task_list = []
design_list = config["design"]
for design in design_list:
    # hg = load_design(benchmark, design, b_pth, use_vir)
    # hg_lst.append(hg)
    task_list.append(pool.apply_async(load_design, args=(benchmark, design, b_pth, False)))
pool.close()
pool.join()
# hg_list = []
for task in task_list:
    hg = task.get()
    hg_list.append(hg)
par_file_list = []
for hg in hg_list:
    par_file = os.path.join(hg.design_pth,'shmetis', hg.design + ".hg.2.5")
    par_file_list.append(par_file)
    
hg_set = HypergraphDataset(benchmark, hg_list, par_file_list, verbose=True)

adaptec1
read ../res/ispd2005/adaptec1/adaptec1.hg.dire
read ../res/ispd2005/adaptec1/adaptec1.gp.pl
process
generate graphs
generate adaptec1


/home/wangzhen/miniconda3/envs/ml/lib/python3.10/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


generate labels
save ../data/ispd2005/ispd2005_2_128_dgl_hypergraph.bin
Done saving data into cached files.


/tmp/ipykernel_9069/1306213928.py:136: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  train_idx = random.sample(all_node, int(n * 0.7))
/tmp/ipykernel_9069/1306213928.py:139: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  valid_idx = random.sample(all_node, int(n * 0.2))


In [39]:
g=hg_set[0]

In [40]:
g

Graph(num_nodes=215659, num_edges=662830,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'valid_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32)})

In [41]:
len(g.edata['w'])

662830